<a href="https://colab.research.google.com/github/lmencisoe/seguros/blob/main/Modelo_seguros_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pandas_profiling import ProfileReport
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import joblib
from google.colab import drive
drive.mount('/content/drive')
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score, auc
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import seaborn as sns
import warnings
import re
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/lmencisoe/seguros/main/seguros.csv",sep=";")

In [ ]:
df.head()

,ACTECON,GRUANALI,anti,TAMAEMP,nrotrab,riesgo,categoria,lineanegocio,modular,producto,fechainicio,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,24,4,10,2,37,2,4,1,1,3,2022-01-07T00:00:00.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24,4,10,2,37,2,4,1,2,3,2021-12-26T00:00:00.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24,4,10,2,37,2,4,1,2,3,2021-12-11T00:00:00.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24,4,10,2,37,2,4,1,2,3,2021-04-19T00:00:00.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24,4,10,2,37,2,4,1,2,3,2021-09-15T00:00:00.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df2 = df.copy()
df2 = df[['ACTECON',	'GRUANALI',	'anti',	'TAMAEMP',	'nrotrab',	'riesgo',	'categoria',	'lineanegocio',	'modular',	'producto'	]]
df2 = df2.drop_duplicates()
df2

,ACTECON,GRUANALI,anti,TAMAEMP,nrotrab,riesgo,categoria,lineanegocio,modular,producto
0,24,4,10,2,37,2,4,1,1,3
1,24,4,10,2,37,2,4,1,2,3
8,24,4,10,2,37,2,4,1,2,7
9,14,4,32,2,62,1,5,1,2,3
10,15,6,28,2,102,3,1,1,2,3
...,...,...,...,...,...,...,...,...,...,...
14056,5,13,10,2,91,3,0,3,2,4
14060,5,13,10,2,91,3,0,3,0,4
14092,33,9,7,2,440,2,6,1,0,7
14093,12,4,15,2,37,3,4,2,0,9


In [ ]:
X_total = df2.drop(['producto'], axis=1)
df2['target'] = df2['producto']
Y_total = df2['target'].astype(float)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_total, Y_total, test_size=0.2, random_state=2022)

In [ ]:
cat_features = X_total.select_dtypes(exclude=["float64", "int64"]).columns.to_list()
numeric_features = X_total.select_dtypes(["float64", "int64"]).columns.to_list()

categorical_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('encoder', OneHotEncoder())
            ]
        )    

numeric_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', StandardScaler())
            ]
        )

preprocessor = ColumnTransformer(
            transformers=[
                ('numerical', numeric_transformer, numeric_features),
                ('categorical', categorical_transformer, cat_features)
            ]
        )

In [ ]:

xgb_pipeline = Pipeline(
            [
                ("preprocessor", preprocessor),
                (
                    "classifier", XGBClassifier(),
                ),
            ]
        )
     

In [ ]:

xgb_param_grid = {
    'classifier__learning_rate': np.arange(0.05, 0.5, 0.05),
    'classifier__max_depth': np.arange(2, 5, 1),
    'classifier__n_estimators': np.arange(2, 5, 1),
    'classifier__subsample': np.arange(0.05, 0.5, 0.05)
}

randomized_roc_auc_xgb = RandomizedSearchCV(estimator=xgb_pipeline, 
                                            param_distributions=xgb_param_grid,
                                            n_iter=50,
                                            scoring='roc_auc',
                                            cv=3, 
                                            verbose=1)

with tf.device('/device:GPU:0'): 
  randomized_roc_auc_xgb.fit(X_train, y_train)

print('Best AUC: ', randomized_roc_auc_xgb.best_score_)
#print('Estimator: ', randomized_roc_auc.best_estimator_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best AUC:  nan


In [ ]:
randomized_roc_auc_xgb.best_params_

{'classifier__subsample': 0.3,
 'classifier__n_estimators': 3,
 'classifier__max_depth': 3,
 'classifier__learning_rate': 0.25}

In [ ]:
y_pred_train = randomized_roc_auc_xgb.predict(X_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

         0.0       0.40      0.27      0.32        93
         1.0       0.60      0.63      0.61        62
         2.0       0.00      0.00      0.00         6
         3.0       0.44      0.64      0.52       206
         4.0       1.00      1.00      1.00       207
         5.0       0.68      0.41      0.51       141
         6.0       0.70      0.89      0.78       239
         7.0       0.49      0.52      0.50       162
         8.0       1.00      0.08      0.15        24
         9.0       0.46      0.39      0.43        84
        10.0       0.50      0.02      0.04        44

    accuracy                           0.63      1268
   macro avg       0.57      0.44      0.44      1268
weighted avg       0.63      0.63      0.60      1268



In [ ]:
randomized_roc_auc_xgb.best_score_

nan